In [1]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd
import re

from os.path import basename

In [3]:
path_ref = '../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL'
path_ref_name = 'Thumb-Arm-Atmega-Leon3-RiscV-KERNEL-DECIMAL-'

In [4]:
# DataFrame name json file
nameFile = []
pathFile = []
rootFile = []

path = os.getcwd()
for root, dirs, files in os.walk(path_ref):
    for file in files:
        if file.endswith(".json"):
            nameFile.append(file)
            pathFile.append(os.path.join(root, file))
            rootFile.append(root)
            
nameFileS = pd.Series(nameFile)
pathFileS = pd.Series(pathFile)
rootFileS = pd.Series(rootFile)

pd.options.display.max_colwidth = 100
print('nameFileS:'+nameFileS)
print('pathFileS:'+pathFileS)
print('rootFileS:'+rootFileS)

dfNF = pd.concat([nameFileS, pathFileS, rootFileS], axis=1)
dfNF.columns = ['FILE_NAME', 'FILE_PATH', 'FILE_ROOT']

0    nameFileS:parameters.json
1    nameFileS:parameters.json
2    nameFileS:parameters.json
3    nameFileS:parameters.json
4    nameFileS:parameters.json
5    nameFileS:parameters.json
6    nameFileS:parameters.json
7    nameFileS:parameters.json
8    nameFileS:parameters.json
dtype: object
0    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/f...
1    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/b...
2    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/b...
3    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/b...
4    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/s...
5    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/k...
6    pathFileS:../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLID

In [5]:
nameFileTim = [] # Timing
pathFileTim = [] # Timing
rootFileTim = [] # Timing

# path = os.getcwd()
for root, dirs, files in os.walk(path_ref):
    for file in files:
        if file.endswith("CC4CSFrameworkExecutionTime.csv"):
            nameFileTim.append(file)
            pathFileTim.append(os.path.join(root, file))
            rootFileTim.append(root)


In [6]:
pathFileTim

['../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/floydwarshall/Leon3/double/optO5-Ofast/32x32/CC4CSFrameworkExecutionTime.csv',
 '../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/floydwarshall/Leon3/double/optO2-09/32x32/CC4CSFrameworkExecutionTime.csv',
 '../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/floydwarshall/Leon3/double/optO3-05/32x32/CC4CSFrameworkExecutionTime.csv',
 '../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/floydwarshall/Leon3/double/optO6-Og/32x32/CC4CSFrameworkExecutionTime.csv',
 '../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/floydwarshall/Leon3/double/optO4-Os/32x32/CC4CSFrameworkExecutionTime.csv',
 '../../../SLIDE-x-D003-Arm-Thumb-Atmega-Leon3-RiscV-KERNEL/SLIDE-x-RES/KERNEL_DECIMAL/floydwarshall/Leon3/double/optO2-05/32x32/CC4CSFrameworkExecutionTime.csv',
 '../../../SLIDE-x-

In [7]:
# Timing file
# Creating Empty DataFrame and Storing it in variable df
dfTimTemp = pd.DataFrame()
dfTimTot = pd.DataFrame()

for i in range(0, len(pathFileTim)):
    path = pathFileTim[i]
    try:
        dfTimTempNew = pd.read_csv(path, sep=',',engine='python', header=None)
    except:
        print(path)
        
    if os.name == 'nt':
        spR = rootFileTim[i].split('\\')
    else:
        spR = rootFileTim[i].split('/')
        
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # Bambu (HW)
    board = spR[-4] # SW
    function = spR[-5]
    dfTimTemp['DEVICE'] = device
    dfTimTemp['BOARD'] = board
    dfTimTemp['FUNCTION'] = function
    dfTimTemp['DATA_TYPE'] = datatype
    dfTimTemp['OPT_FLAG'] = optFlag
    dfTimTemp['MV_SIZE'] = matrixVectorSize
    
    # dfTimTemp['inputName'] = dfTimTemp['inputName'].map(lambda x: x.lstrip('values_'))
    # dfTimTemp['inputName'] = dfTimTemp['inputName'].astype(int)
    # dfTimTemp.sort_values('inputName')
    
    # Filter the dataframe to get rows where the first column matches the search string
    filtered_df = dfTimTempNew[dfTimTempNew[0] == 'InputGenerationTime']
    dfTimTemp['InputGenerationTime'] = filtered_df[1].tolist()
    
    filtered_df = dfTimTempNew[dfTimTempNew[0] == 'ProfilingTime']
    dfTimTemp['ProfilingTime'] = filtered_df[1].tolist()
    
    filtered_df = dfTimTempNew[dfTimTempNew[0] == 'SimulationTime']
    dfTimTemp['SimulationTime'] = filtered_df[1].tolist()

    filtered_df = dfTimTempNew[dfTimTempNew[0] == 'FramaCExecutionTime']
    dfTimTemp['FramaCExecutionTime'] = filtered_df[1].tolist()
    
    filtered_df = dfTimTempNew[dfTimTempNew[0] == 'TotalExecutionTime']
    dfTimTemp['TotalExecutionTime'] = filtered_df[1].tolist()
    
    if i == 0:
        dfTimTot = dfTimTemp
    else:
        frames = [dfTimTot, dfTimTemp]
        dfTimTot = pd.concat(frames)


In [8]:
dfTimTot

,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,InputGenerationTime,ProfilingTime,SimulationTime,FramaCExecutionTime,TotalExecutionTime
0,Leon3,Leon3,floydwarshall,double,optO2-09,32x32,0.244592,19.152312,8.585523,116.451468,1275.856730
0,Leon3,Leon3,floydwarshall,double,optO2-09,32x32,0.244592,19.152312,8.585523,116.451468,1275.856730
0,Leon3,Leon3,floydwarshall,double,optO3-05,32x32,0.244592,19.817784,9.435151,116.451468,1452.753889
0,Leon3,Leon3,floydwarshall,double,optO6-Og,32x32,0.244592,18.398146,8.051783,116.451468,1537.705941
0,Leon3,Leon3,floydwarshall,double,optO4-Os,32x32,0.244592,19.425999,8.536671,116.451468,1480.773421
...,...,...,...,...,...,...,...,...,...,...,...
0,RiscV,RiscV,gcd,float,optO2-08,32x32,0.036471,19.331922,211.023021,90.352404,3091.377318
0,RiscV,RiscV,gcd,float,optO1-01,32x32,0.036471,18.964303,210.293489,90.352404,549.756513
0,RiscV,RiscV,gcd,float,optO3-02,32x32,0.036471,19.537505,210.745240,90.352404,4014.363979
0,RiscV,RiscV,gcd,float,optO1-00,32x32,0.036471,18.833687,211.212565,90.352404,320.451574


In [9]:
dfTimTot.to_csv("TotalTiming.csv", sep=';', index=False, encoding='utf-8')

In [14]:
import pandas as pd

# Paths to the CSV files
csv1_path = 'dataset/xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-1.csv'
csv2_path = 'dataset/xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-2.csv'

# Load the CSV files into DataFrames
df1 = pd.read_csv(csv1_path)
df2 = pd.read_csv(csv2_path, sep=';',engine='python', header=0, names=df1.columns)

# Concatenate the DataFrames along the rows (axis=0)
concatenated_df = pd.concat([df1, df2], ignore_index=True)

# Path to save the concatenated CSV file
merged_csv_path = 'xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-MERGED.csv'

# Save the concatenated DataFrame to a new CSV file
concatenated_df.to_csv(merged_csv_path, index=False, encoding='utf-8')

print(f'Concatenated DataFrame saved to: {merged_csv_path}')

Concatenated DataFrame saved to: xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-MERGED.csv
